In [1]:
%cd /home/jaeheonshim/music-vibes

/home/jaeheonshim/music-vibes


In [2]:
import torch
import numpy as np
import pandas as pd
from torch import nn
from torch.optim.lr_scheduler import CosineAnnealingLR
import torch.nn.functional as F
from torch.utils.data import random_split, DataLoader, Subset
import librosa
from tqdm import tqdm
from torchvggish import vggish_input

from vibenet.dataset import FMAWaveformDataset
from vibenet.models.teacher import PANNsMLP
from vibenet import labels

In [3]:
device = 'cpu'

model = PANNsMLP()
model = model.to(device)
data = torch.load('checkpoints/pretrained_PANN_v3.pt')
model.load_state_dict(data['state_dict'])

<All keys matched successfully>

In [4]:
def eval(y):
    model.eval()

    with torch.inference_mode():
        pred = model(torch.tensor(y).unsqueeze(0).to(device))
        
    pred['acousticness'] = F.sigmoid(pred['acousticness'])
    pred['instrumentalness'] = F.sigmoid(pred['instrumentalness'])
    pred['liveness'] = F.sigmoid(pred['liveness'])
    pred['speechiness'] = torch.clamp(pred['speechiness'], 0, 1)
    pred['danceability'] = torch.clamp(pred['danceability'], 0, 1)
    pred['energy'] = torch.clamp(pred['energy'], 0, 1)
    pred['valence'] = torch.clamp(pred['valence'], 0, 1)

    return pred

In [6]:
y, sr = librosa.load("data/samples/17. Pharrell Williams - Happy.mp3", sr=32000)

pred = eval(y)

for i, label in enumerate(labels):
    print(f"{label:<20}{pred[label].item()}")

acousticness        0.3377591073513031
danceability        0.6294111609458923
energy              0.7161068916320801
instrumentalness    0.028362633660435677
liveness            0.20607727766036987
speechiness         0.1286027431488037
valence             0.8216466307640076
